# **MARSIS REST API Downloader**
Author: g.nodjoumi@jacobs-university.de 

## What is it?
This notebook can be used to download search and download single to multiple MARSIS acquisition (IMG+lbl file) from ODE-PDS.

## How-TO

* Customize [User input](#User-input) by defining:
    * destination folder
    * SHARAD US SCS download (default FALSE) #
    * bounding box (left, bottom, right, top)
    * data type (EDR, RDRAIS, RDRSS)
    * Edit WCS_url and bmap_layerid if custom WCS is available
* Execute until [Track footprints](#Track-footprints) and check available tracks
* To download only specific tracks, edit **user_list** in [Track filter](#Track-filter)
* Continue execution until [Start Download](#Start-download) to start the download    

**Funding**: This study is within the Europlanet 2024 RI and EXPLORE project, and it has received funding from the European Union’s Horizon 2020 research and innovation programme under grant agreement No 871149 and No 101004214.

**Derived from https://github.com/europlanet-gmap/exemplary-notebooks/blob/main/pds/01_search-download_from_ode-pds/01_search_data_products.ipynb**

In [ ]:
import gpt #Clone and install https://github.com/chbrandt/gpt
from gpt.search import ode
import geoviews as gv
gv.extension('bokeh', 'matplotlib')
import os
import rioxarray as riox
from tqdm import tqdm
from utils.download_utils import chunk_creator, download, download_checker, user_filter,downloader_basemap_prep, get_products, geom_splitter
from utils.utils import get_poly

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning) # Only temporary to suppress cartopy and shapely futurewarnings

In [ ]:
MARSISsets = ode.available_datasets('mars')
MARSISsets.loc['MEX'].loc['MARSIS']

In [ ]:
def parallel_df(files, ddir, jobs):
    from joblib import Parallel, delayed
    results = Parallel (n_jobs=jobs)(delayed(download)(files[i], ddir)
                            for i in range(len(files)))    

## User input

In this section, user must select the destination folder ***ddir***, the data type ***Data***, the bounding box coordinates in -180+180 reference system (***min_Lon***, ***min_Lat***, ***max_Lat***, ***max_Lon***), and, if necessary change the ***wcs_url*** and ***bmap_layerid***.

In [ ]:
ddir = './Data/' # Name of the folder where to download files
os.makedirs(ddir, exist_ok=True)
download_simulations = False # Download SHARAD US Surface Clutter Simulation 
Data = 'EDR' # EDR, RDRAIS, RDRSS
min_Lon = -10
min_Lat = -10
max_Lat = 10
max_Lon = 10
######## edit if you want to use custom WCS service
wcs_url ='https://explore.jacobs-university.de/geoserver/ows?service=WCS' # Temporary default WCS service. Is it possible to replace with custom one.
bmap_layerid = 'Mars_Viking_MDIM21_ClrMosaic_global_232m_cog' # IF wcs_url is not the default one, change the layer name accordingly 

######## edit only if plot visualization problems occurs
plot_height = 720

## Track footprints

In following section:
* ***get_products***: use gpt python package combined with the bounding box and data type to access PDS data archive and retreive a geopackage containing all available data information. If the *download_simulations* variable is True, a list of SHARAD USGS SCS simulations will be retrieved.
* ***downloader_basemap_prep***: use the footprint of the geopackage to download the basemap from WCS service. Then creates the geoviews plots.
* ***get_poly***: create a plot-compatible polygon of the user bounding box.
* ***geom_splitter***: split the geometries composed by multiple geometries 
* ***rgb.opts***: plot the basemap, the user bounding box, and all available tracks.

In [ ]:
file_list, track_gdf = get_products([min_Lon, min_Lat, max_Lon, max_Lat], 'MRO', 'SHARAD', Data, download_simulations)
track_gdf = track_gdf.drop_duplicates('geometry')
tracks_geom = geom_splitter(track_gdf)
try:
    bmap_name, ximg, plot_width, rgb = downloader_basemap_prep(wcs_url, bmap_layerid, ddir, track_gdf, plot_height, resx=0.05,resy=0.05)
except Exception as e:
    print(e)
    bmap_name = None
    pass

poly = get_poly([min_Lon, min_Lat, max_Lon, max_Lat])
if bmap_name == None:
    layout = gv.Polygons(poly).opts(alpha=0.2)*gv.Path([gv.Shape(geom) for geom in track_gdf.geometry]).opts( height=plot_height,show_grid=True, alpha=0.5, color = 'red')
else:
    layout = rgb.opts(height=plot_height, width = plot_width*2,show_grid=True).opts(toolbar='left')*gv.Polygons(poly).opts(alpha=0.2)*gv.Path([gv.Shape(geom) for geom in track_gdf.geometry]).opts( height=plot_height,show_grid=True, alpha=0.5, color = 'red')
    
layout

***Notice***: due to MARSIS track's lenght, the plot may take few minutes to compute.

## **Track length filtered**

### **Skip to next block if no need to filter by track length**

In the following section, user can define a maximum length of the track (in degrees), to filter the available tracks. Then a new basemap is downloaded and plotted with the filtered tracks.

In [ ]:
max_length = 20
#degrees - default 179

In [ ]:
 if plot_width <100:
        
        import math
        plot_height = math.ceil((plot_height*100/plot_width))
        #plot_width = math.ceil((src_width*plot_height/src_height))
        #plot_width=100

In [ ]:
length_filtered = track_gdf[track_gdf['geometry'].length <=max_length].drop_duplicates(subset='pdsid',keep='last')
length_filtered.drop('notes', inplace=True, axis=1)
if len(length_filtered)==0:
    print('Dataframe is empty, increase max_length')
else:
    length_filtered.to_file(f'{ddir}/MARSIS_filtered_tracks.gpkg', driver='GPKG')
    filtered_tracks = [pdsid.lower().split('_rgram')[0] for pdsid in length_filtered['pdsid']]
    filtered_list=user_filter(filtered_tracks,file_list)
    bmap_name, ximg, plot_width, rgb = downloader_basemap_prep(wcs_url, bmap_layerid, ddir, length_filtered, plot_height, resx=0.05,resy=0.05)    
    tracks_geom = geom_splitter(length_filtered)
    if plot_width <200:
        plot_width*=5

In [ ]:
if bmap_name == None:
    layout = gv.Polygons(poly).opts(alpha=0.2)*gv.Path(length_filtered).opts( height=plot_height,show_grid=True, alpha=0.5, color = 'red')
else:
    layout = rgb.opts(height=plot_height, width = plot_width*2,show_grid=True).opts(toolbar='left')*gv.Polygons(poly).opts(alpha=0.2)*gv.Path(length_filtered).opts( height=plot_height,show_grid=True, alpha=0.5, color = 'red')
layout

## **Track name filter**

In the following section, available tracks names are printed out and user can filter further the available tracks list by writing desired tracks names in the ***track_names*** list. e.g. '0311401_001','5540401_001'

In [ ]:
available_tracks = list(set([os.path.splitext(os.path.basename(file))[0].split('_')[1] for file in filtered_list]))
print(f'Available track names: {available_tracks}')

In [ ]:
track_names = [] #Enter here the name of selected tracks to download. e.g. *s_00311401*. **IF empty, all tracks will be downloaded**
len(track_names)

## **Track download preparation**

In [ ]:
if len(track_names)>0:
    final_list=user_filter(track_names,filtered_list)
else:
    final_list=filtered_list
dlist = download_checker(final_list,ddir)
chunks, jobs = chunk_creator(dlist)
len(final_list)

## Start download

In [ ]:
with tqdm(total=len(dlist),
         desc = 'Generating files',
         unit='File') as pbar:
    for i in range(len(chunks)):
        files = chunks[i]    
        # print(files)
        parallel_df(files,ddir, jobs)        
        pbar.update(jobs)